In [42]:
import numpy as np                     

import pandas as pd
pd.set_option('mode.chained_assignment', None)      # To suppress pandas warnings.
pd.set_option('display.max_colwidth', None)         # To display all the data in each column
pd.set_option('display.max_columns', None)          # To display every column of the dataset in head()

import warnings
warnings.filterwarnings('ignore')                   # To suppress all the warnings in the notebook.

In [43]:
cust=pd.read_csv('existing_base_train.csv')

In [44]:
cust.shape

(8124, 32)

In [45]:
cust.Revenue_Grid.value_counts()

2    7264
1     860
Name: Revenue_Grid, dtype: int64

# Feature Engineering 

In [46]:
def home_stat(g): 
    if g =='Own Home':
        return 'Owner'
    elif g == 'Rent Privately':
        return 'Rent'
    elif g == 'Rent from Council/HA':
        return 'Rent'
    elif g == 'Live in Parental Hom':
        return 'Rent'
    elif g in 'Unclassified':
        return 'Rent'
working=['Professional','Manual Worker','Business Manager','Secretarial/Admin','Other']
non_working=['Retired','Housewife','Student']
def occ(g):
    if (g.occupation in working) & (g.occupation_partner in working):
        return 'both_working'
    elif (g.occupation in non_working) & (g.occupation_partner in non_working):
        return 'both_non_working'
    else:
        return 'single'
def self_emp(g):
    if (g.self_employed == 'Yes') & (g.self_employed_partner == 'Yes'):
        return 'both_yes'
    elif (g.self_employed == 'No') & (g.self_employed_partner == 'No'):
        return 'both_no'
    else:
        return 'single'

def feature_engg():
    cust['children']=cust['children'].replace('Zero','0')
    cust['children']=cust['children'].replace('4+','4')
    cust['family_income']=cust['family_income'].replace(['>=35,000','<22,500, >=20,000','<25,000, >=22,500','<30,000, >=27,500', '<20,000, >=17,500','<27,500, >=25,000','< 4,000', '<15,000, >=12,500', '<17,500, >=15,000','< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000'],['35000+','20,000-22,500','22,500-25,000','27,500-27,500','17,500-20,000','25,000-27,500','0-4,000','12,500-15,000','15,000-17,500','4,000-8,000','10,000-12,500','8,000-10,000'])
    cust['total_invest']=cust.Investment_in_Mutual_Fund+cust.Investment_Tax_Saving_Bond+cust.Portfolio_Balance
    cust['total_loan']=cust.Personal_Loan+cust.Home_Loan
    cust['total_isurance']=cust.Life_Insurance+cust.Medical_Insurance
    cust['home_status_new']=cust['home_status'].apply(home_stat)
    cust['occupation_Status_new']=cust.apply(occ,axis=1)
    cust['self_employed_new']=cust.apply(self_emp,axis=1)
feature_engg()
cust.shape

(8124, 38)

In [47]:
cust.shape

(8124, 38)

In [48]:
cust_new=cust[['REF_NO','children','age_band','status','family_income','post_area','Average_Credit_Card_Transaction','Balance_Transfer','Term_Deposit', 'total_isurance','Average_A/C_Balance','Online_Purchase_Amount','gender', 'region','total_invest', 'total_loan', 'home_status_new','occupation_Status_new', 'self_employed_new','Revenue_Grid']]

# One hot encoding

In [49]:
def encoding_stat(cust_new):
    stat=pd.get_dummies(cust_new.status)
    stat.head(10)
    stat.drop('Unknown',axis=1,inplace=True)
    cust_new=pd.concat([cust_new,stat],axis=1)
    return cust_new
encoding_stat(cust_new)

def encoding(cust_new):
    cat_columns=['home_status_new','occupation_Status_new','self_employed_new']
    cat_dum=pd.get_dummies(cust_new[cat_columns],drop_first=True)
    cust_new=pd.concat([cust_new,cat_dum],axis=1).head()
    return cust_new
encoding(cust_new)

,REF_NO,children,age_band,status,family_income,post_area,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,total_isurance,Average_A/C_Balance,Online_Purchase_Amount,gender,region,total_invest,total_loan,home_status_new,occupation_Status_new,self_employed_new,Revenue_Grid,home_status_new_Rent,occupation_Status_new_both_working,occupation_Status_new_single,self_employed_new_both_yes,self_employed_new_single
0,5466,2,31-35,Partner,35000+,M51,26.98,29.99,312.25,388.51,108.85,7.68,Female,North West,503.70,230.87,Owner,both_working,both_no,2,0,1,0,0,0
1,9091,0,45-50,Partner,35000+,L40,35.98,74.48,0.00,110.95,48.45,18.99,Female,North West,89.22,15.99,Owner,both_working,both_no,2,0,1,0,0,0
2,9744,1,36-40,Partner,"20,000-22,500",TA19,0.00,24.46,0.00,18.44,0.00,0.00,Female,South West,24.96,0.02,Rent,both_working,both_yes,2,1,1,0,1,0
3,10700,2,31-35,Partner,"22,500-25,000",FK2,44.99,0.00,0.00,29.99,0.00,0.00,Female,Scotland,68.98,0.00,Owner,both_working,both_no,2,0,1,0,0,0
4,1987,0,55-60,Partner,35000+,LS23,0.00,0.00,0.00,0.00,0.00,0.00,Female,Unknown,11.86,0.00,Owner,single,both_no,2,0,0,1,0,0


In [50]:
cust_new.drop(['home_status_new','occupation_Status_new','self_employed_new','status'],axis=1,inplace=True)

In [51]:
def encoding_gender(cust_new):
    gender=pd.get_dummies(cust_new.gender)
    #gender.head(10)
    gender.drop('Unknown',axis=1,inplace=True)
    cust_new=pd.concat([cust_new,gender],axis=1)
    return cust_new
encoding_gender(cust_new)

,REF_NO,children,age_band,family_income,post_area,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,total_isurance,Average_A/C_Balance,Online_Purchase_Amount,gender,region,total_invest,total_loan,Revenue_Grid,Female,Male
0,5466,2,31-35,35000+,M51,26.98,29.99,312.25,388.51,108.85,7.68,Female,North West,503.70,230.87,2,1,0
1,9091,0,45-50,35000+,L40,35.98,74.48,0.00,110.95,48.45,18.99,Female,North West,89.22,15.99,2,1,0
2,9744,1,36-40,"20,000-22,500",TA19,0.00,24.46,0.00,18.44,0.00,0.00,Female,South West,24.96,0.02,2,1,0
3,10700,2,31-35,"22,500-25,000",FK2,44.99,0.00,0.00,29.99,0.00,0.00,Female,Scotland,68.98,0.00,2,1,0
4,1987,0,55-60,35000+,LS23,0.00,0.00,0.00,0.00,0.00,0.00,Female,Unknown,11.86,0.00,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,6516,3,36-40,"17,500-20,000",L33,0.00,0.00,0.00,0.00,0.00,0.00,Male,North West,15.23,0.00,2,0,1
8120,5897,0,61-65,"4,000-8,000",DE6,0.00,0.00,9.49,0.00,2.99,0.00,Female,East Midlands,115.18,0.00,2,1,0
8121,6130,1,41-45,"4,000-8,000",BT28,0.00,107.42,0.00,47.41,38.95,0.00,Female,Northern Ireland,208.43,3.72,2,1,0
8122,980,0,61-65,"0-4,000",CT10,0.00,59.48,0.00,0.00,0.00,0.00,Female,South East,-9.19,0.00,2,1,0


In [52]:
cust_new.drop('gender',axis=1,inplace=True)

In [53]:
#label encoding for age_band
ageband_map={
                'Unknown':0, '18-21':1, '22-25':2,'26-30':3, '31-35':4,'36-40':5,'41-45':6, '45-50':7,
                 '51-55':8,'55-60':9, '61-65':10, '65-70':11,'71+':12 
}
cust_new['ageband_ordinal']=cust_new.age_band.map(ageband_map)

In [54]:
cust_new.drop('age_band',axis=1,inplace=True)

In [55]:
#label encoding for age_band
family_income_map={
                'Unknown':0, '0-4,000':1, '4,000-8,000':2,'8,000-10,000':3, '10,000-12,500':4,'12,500-15,000':5,'15,000-17,500':6, '17,500-20,000':7,
                 '20,000-22,500':8,'22,500-25,000':9, '25,000-27,500':10, '27,500-27,500':11,'35000+':12 
}
cust_new['family_income_ordinal']=cust_new.family_income.map(family_income_map)

In [56]:
cust_new.drop('family_income',axis=1,inplace=True)

In [57]:
#we can replace with its value counts as no one is same number
cust_new.region.value_counts()

South East          1680
North West          1517
Unknown              866
South West           769
West Midlands        658
East Midlands        623
Scotland             615
North                460
Wales                437
East Anglia          344
Northern Ireland     135
Isle of Man           15
Channel Islands        5
Name: region, dtype: int64

In [58]:
reg=cust_new.groupby('region').size()/len(cust_new)
cust_new.loc[:,'region_encoded']=cust_new.region.map(reg)
# try with just value count if F1 changes

In [59]:
cust_new.drop('post_area',axis=1,inplace=True)

In [60]:
cust_new.drop('region',axis=1,inplace=True)

In [61]:
independent_cols=['REF_NO', 'children', 'Average_Credit_Card_Transaction',
       'Balance_Transfer', 'Term_Deposit', 'total_isurance',
       'Average_A/C_Balance', 'Online_Purchase_Amount', 'total_invest',
       'total_loan', 'ageband_ordinal',
       'family_income_ordinal']# removed , 'region_encoded'
X=cust_new[independent_cols] 
y=cust_new.Revenue_Grid

In [62]:
X.shape

(8124, 12)

In [63]:
y.shape

(8124,)

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [65]:
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
X_train[independent_cols] = robust.fit_transform(X_train)
X_test[independent_cols] = robust.transform(X_test)

In [66]:
from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression()

In [67]:
#predicting on train data
y_pred_train = logreg.predict(X_train)

#predicting on test data
y_pred_test = logreg.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score   
pred = logreg.predict(X_test)
print('Accuracy score for test data is:', accuracy_score(y_test,pred))

Accuracy score for test data is: 0.9378461538461539


In [69]:
from sklearn.metrics import f1_score    
f1_score(y_test,y_pred_test)

0.6552901023890785

Random Forest

In [78]:
from sklearn.ensemble import RandomForestClassifier  
model = RandomForestClassifier(random_state = 0)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [79]:
y_pred_test = model.predict(X_test)

In [80]:
print('Accuracy score for test data is:', accuracy_score(y_test,y_pred_test))

Accuracy score for test data is: 0.9476923076923077


In [81]:
f1_score(y_test,y_pred_test)

0.7266881028938906

# Test data 

In [70]:
final_test=pd.read_csv('existing_base_test.csv')

In [71]:
#y_pred_final=model.predict(final_test)
def home_stat(g):
    if g =='Own Home':
        return 'Owner'
    elif g == 'Rent Privately':
        return 'Rent'
    elif g == 'Rent from Council/HA':
        return 'Rent'
    elif g == 'Live in Parental Hom':
        return 'Rent'
    elif g in 'Unclassified':
        return 'Rent'
working=['Professional','Manual Worker','Business Manager','Secretarial/Admin','Other']
non_working=['Retired','Housewife','Student']
def occ(g):
    if (g.occupation in working) & (g.occupation_partner in working):
        return 'both_working'
    elif (g.occupation in non_working) & (g.occupation_partner in non_working):
        return 'both_non_working'
    else:
        return 'single'
def self_emp(g):
    if (g.self_employed == 'Yes') & (g.self_employed_partner == 'Yes'):
        return 'both_yes'
    elif (g.self_employed == 'No') & (g.self_employed_partner == 'No'):
        return 'both_no'
    else:
        return 'single'

def feature_engg():
    final_test['children']=final_test['children'].replace('Zero','0')
    final_test['children']=final_test['children'].replace('4+','4')
    final_test['family_income']=final_test['family_income'].replace(['>=35,000','<22,500, >=20,000','<25,000, >=22,500','<30,000, >=27,500', '<20,000, >=17,500','<27,500, >=25,000','< 4,000', '<15,000, >=12,500', '<17,500, >=15,000','< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000'],['35000+','20,000-22,500','22,500-25,000','27,500-27,500','17,500-20,000','25,000-27,500','0-4,000','12,500-15,000','15,000-17,500','4,000-8,000','10,000-12,500','8,000-10,000'])
    final_test['total_invest']=final_test.Investment_in_Mutual_Fund+final_test.Investment_Tax_Saving_Bond+final_test.Portfolio_Balance
    final_test['total_loan']=final_test.Personal_Loan+final_test.Home_Loan
    final_test['total_isurance']=final_test.Life_Insurance+final_test.Medical_Insurance
    final_test['home_status_new']=final_test['home_status'].apply(home_stat)
    final_test['occupation_Status_new']=final_test.apply(occ,axis=1)
    final_test['self_employed_new']=final_test.apply(self_emp,axis=1)
feature_engg()
final_test.shape


(2031, 37)

In [72]:
final_test_new=final_test[['REF_NO','children','age_band','status','family_income','post_area','Average_Credit_Card_Transaction','Balance_Transfer','Term_Deposit', 'total_isurance','Average_A/C_Balance','Online_Purchase_Amount','gender', 'region','total_invest', 'total_loan', 'home_status_new','occupation_Status_new', 'self_employed_new']]

In [73]:
def encoding_stat(final_test_new):
    stat=pd.get_dummies(final_test_new.status)
    stat.head(10)
    stat.drop('Unknown',axis=1,inplace=True)
    final_test_new=pd.concat([final_test_new,stat],axis=1)
    return final_test_new
encoding_stat(final_test_new)
def encoding(final_test_new):
    cat_columns=['home_status_new','occupation_Status_new','self_employed_new']
    cat_dum=pd.get_dummies(final_test_new[cat_columns],drop_first=True)
    final_test_new=pd.concat([final_test_new,cat_dum],axis=1).head()
    return final_test_new
encoding(final_test_new)
final_test_new.drop(['home_status_new','occupation_Status_new','self_employed_new','status'],axis=1,inplace=True)
def encoding_gender(final_test_new):
    gender=pd.get_dummies(final_test_new.gender)
    #gender.head(10)
    gender.drop('Unknown',axis=1,inplace=True)
    final_test_new=pd.concat([final_test_new,gender],axis=1)
    return final_test_new
encoding_gender(final_test_new)
final_test_new.drop('gender',axis=1,inplace=True)#label encoding for age_band
ageband_map={
                'Unknown':0, '18-21':1, '22-25':2,'26-30':3, '31-35':4,'36-40':5,'41-45':6, '45-50':7,
                 '51-55':8,'55-60':9, '61-65':10, '65-70':11,'71+':12 
}
final_test_new['ageband_ordinal']=final_test_new.age_band.map(ageband_map)
final_test_new.drop('age_band',axis=1,inplace=True)
#label encoding for family income
family_income_map={
                'Unknown':0, '0-4,000':1, '4,000-8,000':2,'8,000-10,000':3, '10,000-12,500':4,'12,500-15,000':5,'15,000-17,500':6, '17,500-20,000':7,
                 '20,000-22,500':8,'22,500-25,000':9, '25,000-27,500':10, '27,500-27,500':11,'35000+':12 
}
final_test_new['family_income_ordinal']=final_test_new.family_income.map(family_income_map)
final_test_new.drop('family_income',axis=1,inplace=True)
reg=final_test_new.groupby('region').size()/len(final_test_new)
final_test_new.loc[:,'region_encoded']=final_test_new.region.map(reg)
independent_cols=['REF_NO', 'children', 'Average_Credit_Card_Transaction',
       'Balance_Transfer', 'Term_Deposit', 'total_isurance',
       'Average_A/C_Balance', 'Online_Purchase_Amount', 'total_invest',
       'total_loan', 'ageband_ordinal',
       'family_income_ordinal']
X=final_test_new[independent_cols]

In [74]:
y_pred_final=logreg.predict(X)

In [75]:
df3 = pd.DataFrame({"Ref_no":X.REF_NO,"Revenue_grid":y_pred_final})

In [76]:
df3.to_csv('cust_classification_output.csv',index=False, header=False)

In [77]:
df3.Revenue_grid.value_counts()

1    2012
2      19
Name: Revenue_grid, dtype: int64

In [84]:
y_pred_final_RF=model.predict(X)## Prediction with Random Forest

In [87]:
df3 = pd.DataFrame({"Ref_no":X.REF_NO,"Revenue_grid":y_pred_final_RF})
df3.to_csv('cust_classification_output.csv',index=False, header=False)
df3.Revenue_grid.value_counts()

2    1637
1     394
Name: Revenue_grid, dtype: int64